In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from newsapi import NewsApiClient
import praw

In [ ]:
# 1. Get stock data
def get_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date)

    # Add technical indicators
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    #df['RSI'] = calculate_rsi(df['Close'], 14)  # Implement RSI calculation
    #df['MACD'] = calculate_macd(df['Close'])  # Implement MACD calculation

    return df

ticker = 'AAPL'
start_date = '2015-01-01'
end_date = '2023-12-31'

#dataset = create_multimodal_dataset(ticker, start_date, end_date)

In [ ]:
stock_data = get_stock_data(ticker, start_date, end_date)

In [ ]:
stock_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_20,SMA_50
Date,,,,,,,,,
2015-01-02 00:00:00-05:00,24.778684,24.789808,23.879987,24.320438,212818400,0.0,0.0,NaN,NaN
2015-01-05 00:00:00-05:00,24.089084,24.169166,23.448429,23.635286,257142000,0.0,0.0,NaN,NaN
2015-01-06 00:00:00-05:00,23.699792,23.897772,23.274912,23.637506,263188400,0.0,0.0,NaN,NaN
2015-01-07 00:00:00-05:00,23.846608,24.069058,23.735383,23.968956,160423600,0.0,0.0,NaN,NaN
2015-01-08 00:00:00-05:00,24.298192,24.947745,24.180292,24.889908,237458000,0.0,0.0,NaN,NaN


In [ ]:
stock_data.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_20,SMA_50
Date,,,,,,,,,
2023-12-22 00:00:00-05:00,194.015137,194.243775,191.818335,192.444580,37122800,0.0,0.0,192.512674,184.200591
2023-12-26 00:00:00-05:00,192.454528,192.732856,191.679185,191.897873,28919300,0.0,0.0,192.674702,184.487575
2023-12-27 00:00:00-05:00,191.341219,192.345186,189.949565,191.997269,48087700,0.0,0.0,192.811381,184.779128
2023-12-28 00:00:00-05:00,192.981369,193.498269,192.017156,192.424713,34049900,0.0,0.0,193.020626,185.110401
2023-12-29 00:00:00-05:00,192.742785,193.239801,190.585738,191.380966,42628800,0.0,0.0,193.148856,185.446808


In [ ]:
null_counts = stock_data.isnull().sum()
null_counts

,0
Open,0
High,0
Low,0
Close,0
Volume,0
Dividends,0
Stock Splits,0
SMA_20,19
SMA_50,49


In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2264 entries, 2015-01-02 00:00:00-05:00 to 2023-12-29 00:00:00-05:00
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          2264 non-null   float64
 1   High          2264 non-null   float64
 2   Low           2264 non-null   float64
 3   Close         2264 non-null   float64
 4   Volume        2264 non-null   int64  
 5   Dividends     2264 non-null   float64
 6   Stock Splits  2264 non-null   float64
 7   SMA_20        2245 non-null   float64
 8   SMA_50        2215 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 176.9 KB


In [ ]:
  ohlc_columns = ['Open', 'High', 'Low', 'Close']
  correlation_matrix = stock_data[ohlc_columns].corr()

  print(correlation_matrix)

           Open      High       Low     Close
Open   1.000000  0.999866  0.999838  0.999665
High   0.999866  1.000000  0.999801  0.999839
Low    0.999838  0.999801  1.000000  0.999846
Close  0.999665  0.999839  0.999846  1.000000


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

X = stock_data[ohlc_columns].dropna()

vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i)
                   for i in range(len(X.columns))]

print(vif_data)

  Feature           VIF
0    Open  26320.333573
1    High  30371.347884
2     Low  26196.187305
3   Close  23772.792118


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(stock_data[ohlc_columns].dropna())

print("Explained variance ratio:", pca.explained_variance_ratio_)

Explained variance ratio: [9.99856934e-01 8.43723116e-05]


In [ ]:
q = f"{ticker} stock"

In [ ]:
newsapi = NewsApiClient(api_key='30dda5fa45cb4eb785e84e2f5295f3bd')
articles = newsapi.get_everything(q=ticker, from_param= start_date, to= end_date, language='en')